<a href="https://colab.research.google.com/github/NewCodeLearner/DeepLearning_PyTorch/blob/main/ConvolutionalNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CNN's are neural networks that are used to classify images.

We'll look at the Mnist Dataset and I'll explain a broad overview of how CNN's work.

MNIST Dataset comes with 50000 training set and 10000 test images set

setosa.io/image for demo of CNN

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
# Convert MNIST image file into Tensor of 4-dimensions (# of images, ht,width,color channel)
transform = transforms.ToTensor()

In [13]:
#Train data Download and save the MNIST data into local directory
train_data = datasets.MNIST(root='/cnn_data',train =True,download=True,transform=transform)

In [6]:
test_data = datasets.MNIST(root='/cnn_data',train=False,download=True,transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 34871609.45it/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1187057.86it/s]

Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 8322091.91it/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 5831199.50it/s]

Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [7]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /ccn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [8]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [35]:
# create a small batch size for images
train_loader = DataLoader(train_data,batch_size=10,shuffle=True)
train_loader = DataLoader(test_data,batch_size=10,shuffle=False)

In [36]:
#Define our CNN Model
#Describe convolutional layer and what its doing ( 2 Convolutional layers)
#this is just an exmaple.
conv1= nn.Conv2d(1,6,3,1)  #input size 1,output 6(feature maps),kernel size 3x3 ,stride 1 pixel at a time
conv2= nn.Conv2d(6,16,3,1) #6 output becomes input here , 16 output feature maps

In [37]:
# Grab 1 MNIST image
for i,(X_train,y_train) in enumerate(train_data):
    break

In [38]:
X_train.shape
#torch.Size([1, 28, 28]) 1 image of 28x28 size

torch.Size([1, 28, 28])

In [39]:
x = X_train.view(1,1,28,28) # 1 batch--1image--28x28 pixels

In [40]:
# Perform our first convolution
x=F.relu(conv1(x)) #rectified linear unit for activation function

In [41]:
# 1 single image , 6 the filters we asked for,
#26x26 is the image as we dont have padding and mnist data is all in the middle of the image so we are not losing any data but for other projects padding may be needed.
x.shape

torch.Size([1, 6, 26, 26])

In [42]:
# pass thru pooling layer
x=F.max_pool2d(x,2,2) # 2x2 kernel for pooling stride of 2

In [43]:
#as we set kernel 2x2 the image is now 26/2 =13
x.shape

torch.Size([1, 6, 13, 13])

In [44]:
#Do our second convolution layer
x=F.relu(conv2(x))

In [45]:
x.shape #again we didnt set padding so we lose 2 pixels around the images edges

torch.Size([1, 16, 11, 11])

In [46]:
#pooling layer
x=F.max_pool2d(x,2,2) #11/2 =5.5 but we have to round down, because we cant invent data to round up

In [34]:
x.shape

torch.Size([1, 16, 2, 2])

In [53]:
# Model Class
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1=nn.Conv2d(1,6,3,1)
      self.conv2=nn.Conv2d(6,16,3,1)
      #fully connected layer
      self.fc1= nn.Linear(5*5*16,120)
      self.fc2= nn.Linear(120,84)
      self.fc3= nn.Linear(84,10)

    #forward pass
    def forward(self,x):
      x=F.relu(self.conv1(x))
      x=F.max_pool2d(x,2,2)
      #second pass
      x=F.relu(self.conv2(x))
      x=F.max_pool2d(x,2,2)
      #re-View to flatten out
      x=x.view(-1,5*5*16) # negative one so we can vary the batch size
      #Fully connected layers
      x=F.relu(self.fc1(x))
      x=F.relu(self.fc2(x))
      x=self.fc3(x)
      return F.log_softmax(x,dim=1)

In [55]:
# create an instance of model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [56]:
# Loss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001) #smaller the lr the longer training will take.
